In [1]:
MIN_NB_NON_CENSORED = 40

# SurvSet

In [2]:
from SurvSet.data import SurvLoader
loader = SurvLoader()
# List of available datasets and meta-info
df_ds = loader.df_ds
df_ds = df_ds[~df_ds['is_td']]
df_ds

,ds,is_td,n,n_fac,n_ohe,n_num
0,hdfail,False,52422,5,87,1
1,stagec,False,146,4,15,3
2,veteran,False,137,3,5,3
3,vdv,False,78,0,0,4705
4,AML_Bull,False,116,0,0,6283
...,...,...,...,...,...,...
71,prostate,False,502,6,16,9
72,flchain,False,7874,4,26,6
73,grace,False,1000,2,2,3
74,gse3143,False,158,1,3,8660


In [3]:
excluded = [
    'hdfail',
    'UnempDur',
    'divorce',
    'Unemployment',
    'Rossi',
    'oldmort',
    'scania',
]

In [11]:
from sklearndf.pipeline import PipelineDF
from sklearndf.transformation import OneHotEncoderDF, ColumnTransformerDF, SimpleImputerDF, StandardScalerDF
from sklearn.compose import make_column_selector
from sksurv.util import Surv
import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd


for ds in tqdm(df_ds['ds']):
    if ds in excluded:
        continue
    df, _ = loader.load_dataset(ds_name=ds).values()

    Xcols = []
    for c in df.columns:
        if c in ['pid', 'event', 'time']:
            continue
        if 'num' == c.split('_')[0]:
            Xcols.append(df[[c]])
        elif 'fac' == c.split('_')[0]:
            ohe = pd.get_dummies(df[c]).astype('int8').add_prefix(c.replace('fac_', '')+'_')
            Xcols.append(ohe)
    X = pd.concat(Xcols, axis=1)
    y = Surv.from_arrays(event=np.array(df['event']).astype('bool'), time=df['time'])
    
    y_non_censored = y[y['event']]
    if y_non_censored.shape[0] < MIN_NB_NON_CENSORED:
        print('skipped', ds)
        continue
    print(ds, X.shape[1])
#     res = {'X': X, 'y': y}
    
#     with open('datasets/SurvSet_'+ds+'.pickle', 'wb') as handle:
#         pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|                                                                                                                                                       | 0/69 [00:00<?, ?it/s]

stagec 22
veteran 11


  6%|████████▎                                                                                                                                      | 4/69 [00:01<00:30,  2.13it/s]

skipped vdv


 16%|██████████████████████▋                                                                                                                       | 11/69 [00:04<00:17,  3.40it/s]

AML_Bull 6283
LeukSurv 31
zinc 31
burn 23
nwtco 14
rott2 18


 16%|██████████████████████▋                                                                                                                       | 11/69 [00:04<00:22,  2.58it/s]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

# TCGA

In [5]:
from glob import glob
import pandas as pd

In [6]:
surv = pd.read_excel('/mnt/data/lrebaud/data/TCGA/clinical_TCGA.xlsx')
surv.set_index('bcr_patient_barcode', inplace=True)
surv = surv[['OS', 'OS.time']]
surv = surv.rename(columns={
    'OS': 'event',
    'OS.time': 'time'
})
surv = surv.dropna(how='any')

In [7]:
for f in tqdm(glob('/mnt/data/lrebaud/data/TCGA/RNA-seq/*/tpm.csv')):    
    ds = f.split('/')[-2]
    print(ds)
    gene_annotations = pd.read_csv(f.replace('tpm.csv', 'gene_annotations.csv'))
    gene_annotations = gene_annotations[gene_annotations['gene_type']=='protein_coding']

    tpm = pd.read_csv(f, low_memory=False)
    tpm = tpm.T
    tpm.columns = tpm.iloc[0]
    tpm = tpm.iloc[1:]
    tpm = tpm[tpm['Sample Type'].isin(['Primary Tumor', 'Primary Blood Derived Cancer - Peripheral Blood'])]
    tpm.set_index('Case ID', inplace=True)
    tpm = tpm.drop(columns=['Sample Type'])
    tpm = tpm[~tpm.index.duplicated(keep='first')]
    assert len(tpm.index.unique()) == len(tpm)
    assert len(tpm) > 0

    tpm = tpm[gene_annotations['gene_id']]

    common_patient = list(set(tpm.index)&set(surv.index))
    tpm = tpm.loc[common_patient]
    ds_surv = surv.loc[common_patient]
    tpm = tpm.astype('float64')
    tpm = np.log(tpm+1)
    tpm = tpm.loc[:, (tpm != tpm.iloc[0]).any()]  # drop constant columns
        
    y = Surv.from_arrays(event=np.array(ds_surv['event']).astype('bool'), time=ds_surv['time'])
    y_non_censored = y[y['event']]
    if y_non_censored.shape[0] < MIN_NB_NON_CENSORED:
        print('skipped', ds)
        continue

    res = {'X': tpm, 'y': y}
    
    with open('datasets/TCGA_'+ds+'.pickle', 'wb') as handle:
        pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|                                                                                                                                                       | 0/33 [00:00<?, ?it/s]

HNSC


  3%|████▎                                                                                                                                          | 1/33 [00:36<19:31, 36.60s/it]

DLBC


  6%|████████▋                                                                                                                                      | 2/33 [00:43<09:54, 19.16s/it]

...

CESC


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 32/33 [13:20<00:25, 25.23s/it]

KICH


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [13:28<00:00, 24.49s/it]

skipped KICH
